Starting with the file most recent file (2013 - September 10, 2018). Working with Bainbridge departures only.

In [401]:
import pandas as pd
import numpy as np

laptop_file_path = 'C:/Users/Keith/Google Drive/H Drive Full Backup/Fall 2018/CS 495 - Senior Seminar/Ferry Data/P000413-090518_2013-2018-09-10.csv'
dataset = pd.read_csv(laptop_file_path)

bainbridge_departures = dataset.loc[dataset['departure_terminal'] == 'Bainbridge'].reset_index()
bainbridge_departures = bainbridge_departures.drop('index', axis=1)

In [402]:
bainbridge_departures.head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
0,Bainbridge,Colman,55:00.0,05:56.0,37:37.0,32.0
1,Bainbridge,NaN,45:00.0,NaN,NaN,NaN
2,Bainbridge,Colman,20:00.0,19:01.0,49:40.0,30.0
3,Bainbridge,NaN,20:00.0,NaN,NaN,NaN
4,Bainbridge,NaN,05:00.0,06:16.0,NaN,NaN


<a href = "https://web.archive.org/web/20130123233819/http:/www.wsdot.wa.gov/ferries/pdf/2013winter.pdf"> Winter 2013 Schedule </a>

In [403]:
# Monday-Friday leave Bainbridge Island Winter 2013
# mfw13 means Monday-Friday Winter 2013
mfw13_scheduled_departures_24h = ["0:55:00", "4:45:00", "5:20:00", "6:20:00", "7:05:00", "7:55:00","8:45:00", "9:40:00", 
                            "10:25:00", "11:30:00", "12:20:00", "13:10:00", "14:05:00", "14:55:00", "15:50:00", "16:35:00", 
                            "17:30:00", "18:30:00", "19:10:00", "20:10:00", "20:55:00", "21:45:00", "22:35:00"]

# satw13 means Saturday Winter 2013
satw13_scheduled_departures_24h = ["0:55:00","05:20:00","07:05:00","07:55:00","08:45:00","9:35:00", "10:25:00",
                                  "11:30:00","12:20:00","13:10:00","14:05:00","14:55:00","15:50:00",
                                  "16:35:00","17:30:00","18:30:00","19:10:00","20:10:00","20:55:00",
                                  "21:45:00","22:30:00","0:00:00","1:25:00"]

# sunw13 means Sunday Winter 2013
sunw13_scheduled_departures_24h = ["05:20:00","07:05:00","08:45:00","9:35:00", "10:25:00",
                                  "11:30:00","12:20:00","13:10:00","14:05:00","14:55:00","15:50:00",
                                  "16:35:00","17:30:00","18:30:00","19:10:00","20:10:00","20:55:00",
                                  "21:45:00","22:30:00","0:00:00","1:25:00"]

#wkw13 means Week Winder 2013
wkw13_scheduled_departures_24h = (5*mfw13_scheduled_departures_24h) + satw13_scheduled_departures_24h + sunw13_scheduled_departures_24h

In [404]:
def extractMinutesPastHour(time):
    m = time.split(':')
    return int(float(m[0]))

# For time in format hh:mm:ss this method returns mm
def extractMinutes24h(time):
    m = time.split(':')
    return int(m[1])

In [405]:
import datetime

def generate_timestamps(dataset, scheduled_departures_24h, starting_point, year, month, day):
    #import pdb; pdb.set_trace()

    tdelta = datetime.timedelta(days = 1)

    prevHour = 0

    departure_times = []
    timestamps = []
    weekdays = []

    i = starting_point

    for index, row in dataset.iterrows():
        departure24h = scheduled_departures_24h[i % len(scheduled_departures_24h)]
        hour, minute, seconds = departure24h.split(":")
        scheduled_departure_minutes = extractMinutesPastHour(row.iloc[2])
        scheduled_departure_24h = extractMinutes24h(departure24h)

        if(int(prevHour) > int(hour)):
            d = d + tdelta
            year = d.year
            month = d.month
            day = d.day

        if (not scheduled_departure_minutes == scheduled_departure_24h):
            if(d.weekday() == 1):
                # --- this is a quick fix not sure why this works ----
                d = d - tdelta
                year = d.year
                month = d.month
                day = d.day
                # ------------------------------------

                i = i + 1

                departure24h = scheduled_departures_24h[i % len(scheduled_departures_24h)]
                scheduled_hour, scheduled_minutes, scheduled_seconds = departure24h.split(":")
                scheduled_departure_minutes = extractMinutesPastHour(row.iloc[2])

                departure_times.append(departure24h)
                d = datetime.date(year, month, day)
                timestamps.append("%s %02d:%02d:%02d" % (d, int(scheduled_hour), int(scheduled_minutes), int(scheduled_seconds)))
                weekdays.append(d.weekday())

                prevHour = hour  
                i = i + 1
                continue
                if (not scheduled_departure_minutes == scheduled_minutes):
                    print("ERROR")
                    break

            print("""Was able to match departure times up until this row at index %d:\n%s\n
                    Was trying to insert this timestamp: %s""" % 
                                              (index, row,("%s %02d:%02d:%02d" % (d, int(hour), int(minute), int(seconds)))))
            break

        departure_times.append(departure24h)
        d = datetime.date(year, month, day)
        timestamps.append("%s %02d:%02d:%02d" % (d, int(hour), int(minute), int(seconds)))
        weekdays.append(d.weekday())

        prevHour = hour
        i = i + 1
        
    return timestamps

In [406]:
timestamps = generate_timestamps(bainbridge_departures, wkw13_scheduled_departures_24h, 23, 2013, 1, 1)

Was able to match departure times up until this row at index 3297:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       05:00.0
actual_departure          05:30.0
Actual_Arrival            36:59.0
TransitTime                    31
Name: 3297, dtype: object

                    Was trying to insert this timestamp: 2013-05-27 05:20:00


In [407]:
len(timestamps)

3297

In [408]:
timestamps[3290:]

['2013-05-26 20:10:00',
 '2013-05-26 20:55:00',
 '2013-05-26 21:45:00',
 '2013-05-26 22:30:00',
 '2013-05-27 00:00:00',
 '2013-05-27 01:25:00',
 '2013-05-27 04:45:00']

Want to git rid of those last three dates.

In [409]:
timestamps = timestamps[:3296]

In [410]:
len(timestamps)

3296

May 27, 2013 Starts at index 3296 in `bainbridge_departures`

In [411]:
bainbridge_departures.iloc[3296:3300]

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
3296,Bainbridge,Colman,20:00.0,20:11.0,51:37.0,31.0
3297,Bainbridge,Colman,05:00.0,05:30.0,36:59.0,31.0
3298,Bainbridge,Colman,55:00.0,56:22.0,26:55.0,30.0
3299,Bainbridge,Colman,45:00.0,45:00.0,16:17.0,31.0


The only difference between the <a href = "https://web.archive.org/web/20130123233819/http:/www.wsdot.wa.gov/ferries/pdf/2013winter.pdf"> Winter 2013 Schedule</a> and the <a href = "https://web.archive.org/web/20130624071202/http:/www.wsdot.wa.gov/ferries/pdf/2013Spring.pdf"> Spring 2013 Schedule</a> happend on May 27, 2013

In [412]:
# May 27, 2013
may2713 = ["05:20:00","07:05:00","07:55:00","08:45:00","9:35:00", "10:25:00","11:30:00","12:20:00","13:10:00",
           "14:05:00","14:55:00","15:50:00","16:35:00","17:30:00","18:30:00","19:10:00","20:10:00","20:55:00",
           "21:45:00","22:30:00","0:00:00","1:25:00"]

In [413]:
may2713timestamps = generate_timestamps(bainbridge_departures.iloc[3296:], may2713, 0, 2013, 5, 27)

Was able to match departure times up until this row at index 3319:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       20:00.0
actual_departure          20:00.0
Actual_Arrival            51:12.0
TransitTime                    31
Name: 3319, dtype: object

                    Was trying to insert this timestamp: 2013-05-27 07:55:00


In [414]:
may2713timestamps = may2713timestamps[:22]

In [415]:
timestamps = timestamps + may2713timestamps

This below is the beginning of Tuesday, May 28, 2013

In [416]:
bainbridge_departures.iloc[3318:3322]

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
3318,Bainbridge,Colman,45:00.0,45:56.0,16:44.0,31.0
3319,Bainbridge,Colman,20:00.0,20:00.0,51:12.0,31.0
3320,Bainbridge,Colman,20:00.0,23:59.0,51:35.0,28.0
3321,Bainbridge,Colman,05:00.0,06:51.0,40:22.0,34.0


In [417]:
len(mfw13_scheduled_departures_24h)

23

In [418]:
timestamps_2 = generate_timestamps(bainbridge_departures.iloc[3318:], wkw13_scheduled_departures_24h, 24, 2013, 5, 28)

Was able to match departure times up until this row at index 4154:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       20:00.0
actual_departure          19:33.0
Actual_Arrival            49:12.0
TransitTime                    30
Name: 4154, dtype: object

                    Was trying to insert this timestamp: 2013-07-04 04:45:00


In [419]:
timestamps = timestamps + timestamps_2

In [420]:
len(timestamps)

4154

In [421]:
timestamps[4150:]

['2013-07-03 20:55:00',
 '2013-07-03 21:45:00',
 '2013-07-03 22:35:00',
 '2013-07-04 00:55:00']

The fourth of July schedule is the same as the May 27, 2013 schedule.

Thursday, July 4, 2013

In [422]:
july0413timestamps = generate_timestamps(bainbridge_departures[4154:], may2713, 0, 2013, 7, 4)

Was able to match departure times up until this row at index 4176:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       45:00.0
actual_departure          44:36.0
Actual_Arrival            13:56.0
TransitTime                    29
Name: 4176, dtype: object

                    Was trying to insert this timestamp: 2013-07-05 05:20:00


In [423]:
timestamps = timestamps + july0413timestamps

In [424]:
len(timestamps)

4176

Need to start again this time from Thursday

In [425]:
timestamps_2 = generate_timestamps(bainbridge_departures[4176:], wkw13_scheduled_departures_24h, (23*4) + 1, 2013, 7, 5)

Was able to match departure times up until this row at index 5507:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       05:00.0
actual_departure          05:06.0
Actual_Arrival            32:58.0
TransitTime                    27
Name: 5507, dtype: object

                    Was trying to insert this timestamp: 2013-09-02 05:20:00


In [426]:
timestamps = timestamps + timestamps_2

Monday, September 2, 2013 was a holiday.

In [427]:
timestamps = timestamps[:5506]

In [428]:
len(timestamps)

5506

In [429]:
timestamps[5500:]

['2013-09-01 20:10:00',
 '2013-09-01 20:55:00',
 '2013-09-01 21:45:00',
 '2013-09-01 22:30:00',
 '2013-09-02 00:00:00',
 '2013-09-02 01:25:00']

In [430]:
bainbridge_departures.iloc[5506:].head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
5506,Bainbridge,Colman,20:00.0,20:16.0,49:50.0,29.0
5507,Bainbridge,Colman,05:00.0,05:06.0,32:58.0,27.0
5508,Bainbridge,Colman,55:00.0,55:00.0,26:35.0,31.0
5509,Bainbridge,Colman,45:00.0,45:06.0,14:50.0,29.0
5510,Bainbridge,Colman,35:00.0,35:36.0,06:01.0,31.0


In [431]:
sept0213timestamps = generate_timestamps(bainbridge_departures[5506:], may2713, 0, 2013, 9, 2)

Was able to match departure times up until this row at index 5529:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       20:00.0
actual_departure          20:05.0
Actual_Arrival            53:03.0
TransitTime                    33
Name: 5529, dtype: object

                    Was trying to insert this timestamp: 2013-09-02 07:55:00


In [432]:
sept0213timestamps = sept0213timestamps[:22]

In [433]:
timestamps = timestamps + sept0213timestamps

In [434]:
len(timestamps)

5528

In [435]:
timestamps[5520:]

['2013-09-02 18:30:00',
 '2013-09-02 19:10:00',
 '2013-09-02 20:10:00',
 '2013-09-02 20:55:00',
 '2013-09-02 21:45:00',
 '2013-09-02 22:30:00',
 '2013-09-03 00:00:00',
 '2013-09-03 01:25:00']

In [436]:
bainbridge_departures.iloc[5528:].head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
5528,Bainbridge,Colman,45:00.0,45:07.0,12:15.0,27.0
5529,Bainbridge,Colman,20:00.0,20:05.0,53:03.0,33.0
5530,Bainbridge,Colman,20:00.0,20:00.0,49:57.0,29.0
5531,Bainbridge,Colman,05:00.0,06:03.0,36:39.0,30.0
5532,Bainbridge,Colman,55:00.0,56:43.0,26:53.0,30.0


Tuesday, September 3, 2013 (is the start of whats above)

In [437]:
bainbridge_departures = bainbridge_departures.drop(7185)

In [438]:
timestamps_2 = generate_timestamps(bainbridge_departures[5528:], wkw13_scheduled_departures_24h, (23*1) + 1, 2013, 9, 3)

Was able to match departure times up until this row at index 7471:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       20:00.0
actual_departure          21:09.0
Actual_Arrival            52:53.0
TransitTime                    31
Name: 7471, dtype: object

                    Was trying to insert this timestamp: 2013-11-28 04:45:00


In [439]:
len(timestamps_2)

1942

In [440]:
timestamps = timestamps + timestamps_2

In [441]:
timestamps_2[1935:]

['2013-11-27 18:30:00',
 '2013-11-27 19:10:00',
 '2013-11-27 20:10:00',
 '2013-11-27 20:55:00',
 '2013-11-27 21:45:00',
 '2013-11-27 22:35:00',
 '2013-11-28 00:55:00']

In [442]:
bainbridge_departures[7470:].head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
7471,Bainbridge,Colman,20:00.0,21:09.0,52:53.0,31.0
7472,Bainbridge,Colman,05:00.0,04:41.0,41:21.0,37.0
7473,Bainbridge,Colman,55:00.0,54:33.0,25:12.0,31.0
7474,Bainbridge,Colman,45:00.0,44:43.0,20:07.0,36.0
7475,Bainbridge,Colman,35:00.0,35:00.0,04:01.0,29.0


In [443]:
nov2813timestamps = generate_timestamps(bainbridge_departures[7470:], may2713, 0, 2013, 11, 28)

Was able to match departure times up until this row at index 7493:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       45:00.0
actual_departure          45:58.0
Actual_Arrival            20:01.0
TransitTime                    35
Name: 7493, dtype: object

                    Was trying to insert this timestamp: 2013-11-29 05:20:00


In [444]:
timestamps = timestamps + nov2813timestamps

In [445]:
bainbridge_departures[7492:].head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
7493,Bainbridge,Colman,45:00.0,45:58.0,20:01.0,35.0
7494,Bainbridge,Colman,20:00.0,20:03.0,51:11.0,31.0
7495,Bainbridge,Colman,20:00.0,22:03.0,52:19.0,30.0
7496,Bainbridge,Colman,05:00.0,05:01.0,35:29.0,30.0
7497,Bainbridge,Colman,55:00.0,55:26.0,26:27.0,31.0


The top of the above is Friday, November 29, 2013

In [446]:
timestamps_2 = generate_timestamps(bainbridge_departures[7492:], wkw13_scheduled_departures_24h, (23*4) + 1, 2013, 11, 29)

Was able to match departure times up until this row at index 8079:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       20:00.0
actual_departure          20:15.0
Actual_Arrival            51:40.0
TransitTime                    31
Name: 8079, dtype: object

                    Was trying to insert this timestamp: 2013-12-25 04:45:00


In [447]:
timestamps = timestamps + timestamps_2

In [448]:
bainbridge_departures[8078:].head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
8079,Bainbridge,Colman,20:00.0,20:15.0,51:40.0,31.0
8080,Bainbridge,Colman,05:00.0,06:36.0,38:42.0,32.0
8081,Bainbridge,Colman,55:00.0,55:25.0,24:45.0,29.0
8082,Bainbridge,Colman,45:00.0,45:00.0,15:54.0,30.0
8083,Bainbridge,Colman,35:00.0,37:32.0,11:13.0,34.0


In [449]:
dec2513timestamps = generate_timestamps(bainbridge_departures[8078:], may2713, 0, 2013, 12, 25)

Was able to match departure times up until this row at index 8101:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       45:00.0
actual_departure          44:36.0
Actual_Arrival            14:25.0
TransitTime                    30
Name: 8101, dtype: object

                    Was trying to insert this timestamp: 2013-12-26 05:20:00


In [450]:
timestamps = timestamps + dec2513timestamps

In [451]:
bainbridge_departures[8100:].head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
8101,Bainbridge,Colman,45:00.0,44:36.0,14:25.0,30.0
8102,Bainbridge,Colman,20:00.0,20:53.0,50:24.0,30.0
8103,Bainbridge,Colman,20:00.0,19:40.0,50:15.0,31.0
8104,Bainbridge,Colman,05:00.0,06:06.0,35:48.0,29.0
8105,Bainbridge,Colman,55:00.0,55:29.0,26:33.0,31.0


The top of above is Thursday December 26, 2013

In [452]:
timestamps_2 = generate_timestamps(bainbridge_departures[8100:], wkw13_scheduled_departures_24h, (23*3) + 1, 2013, 12, 26)

Was able to match departure times up until this row at index 8236:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       20:00.0
actual_departure          20:00.0
Actual_Arrival            55:34.0
TransitTime                    35
Name: 8236, dtype: object

                    Was trying to insert this timestamp: 2014-01-01 04:45:00


In [453]:
timestamps = timestamps + timestamps_2

In [454]:
len(timestamps)

8235

In [455]:
bainbridge_departures_2013_with_index = bainbridge_departures[0:8235]
bainbridge_departures_2013_with_index.index = pd.to_datetime(timestamps)

In [456]:
bainbridge_departures_2013_with_index = bainbridge_departures_2013_with_index["2013"]

In [457]:
bainbridge_departures_2013_with_index.head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
2013-01-01 00:55:00,Bainbridge,Colman,55:00.0,05:56.0,37:37.0,32.0
2013-01-01 04:45:00,Bainbridge,NaN,45:00.0,NaN,NaN,NaN
2013-01-01 05:20:00,Bainbridge,Colman,20:00.0,19:01.0,49:40.0,30.0
2013-01-01 06:20:00,Bainbridge,NaN,20:00.0,NaN,NaN,NaN
2013-01-01 07:05:00,Bainbridge,NaN,05:00.0,06:16.0,NaN,NaN


In [458]:
bainbridge_departures_2013_with_index.tail()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
2013-12-31 19:10:00,Bainbridge,Colman,10:00.0,10:03.0,44:26.0,34.0
2013-12-31 20:10:00,Bainbridge,Colman,10:00.0,10:53.0,41:57.0,31.0
2013-12-31 20:55:00,Bainbridge,Colman,55:00.0,55:00.0,26:36.0,31.0
2013-12-31 21:45:00,Bainbridge,Colman,45:00.0,45:17.0,17:09.0,32.0
2013-12-31 22:35:00,Bainbridge,Colman,35:00.0,38:45.0,13:36.0,35.0


In [459]:
bainbridge_departures_2013_with_index.shape

(8234, 6)

In [460]:
timestamps_2013 = bainbridge_departures_2013_with_index.index

<a href = "https://stackoverflow.com/questions/48796729/how-to-add-a-year-to-a-column-of-dates-in-pandas">How to add a year to a column of dates in pandas </a>

In [461]:
timestamps_2014 = timestamps_2013 + pd.offsets.DateOffset(years=1)

The Holdiays in 2014:
- Wednesday, January 1, 2014
- Monday, May 26, 2014
- Friday, July 4, 2014
- Monday, September 1, 2014
- Thursday, November 27, 2014
- Thursday, December 25, 2014

`d.weekday()`: <a href = "https://docs.python.org/2/library/datetime.html">Monday is 0 and Sunday is 6</a>

<a href = "https://stackoverflow.com/questions/17152719/change-date-of-a-datetimeindex">Change date of a DateTimeIndex</a>

<a href = "https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-list-of-lists">How to make a flat list out of list of lists?</a>

In [462]:
def generate_more_timestamps(startingDate, holidays, weekday, saturday, sunday):
    timestamps = [[pd.Timestamp('2014-01-01 00:55:00')]]
    startingYear = startingDate.year
    d = startingDate
    
    
    while(not (d.year == (startingYear + 1))):
        if(d in holidays):
            timestamps.append(saturday[1:].map(lambda t: t.replace(year=d.year, month=d.month, day=d.day)).tolist())
            temp = d + datetime.timedelta(days = 1)
            timestamps.append(sunday[0:2].map(lambda t: t.replace(year=temp.year, month=temp.month, day=temp.day)).tolist())
        else:
            if(d.weekday() == 5):
                timestamps.append(saturday[1:].map(lambda t: t.replace(year=d.year, month=d.month, day=d.day)).tolist())
                temp = d + datetime.timedelta(days = 1)
                timestamps.append(sunday[0:2].map(lambda t: t.replace(year=temp.year, month=temp.month, day=temp.day)).tolist())
            elif(d.weekday() == 6):
                timestamps.append(sunday[2:].map(lambda t: t.replace(year=d.year, month=d.month, day=d.day)).tolist())
                temp = d + datetime.timedelta(days = 1)
                timestamps.append(sunday[0:2].map(lambda t: t.replace(year=temp.year, month=temp.month, day=temp.day)).tolist())
            else:
                timestamps.append(weekday[1:].map(lambda t: t.replace(year=d.year, month=d.month, day=d.day)).tolist())
                temp = d + datetime.timedelta(days = 1)
                timestamps.append(weekday[0:1].map(lambda t: t.replace(year=temp.year, month=temp.month, day=temp.day)).tolist())
        
        d = d + datetime.timedelta(days = 1)
        
    return [item for sublist in timestamps for item in sublist]

In [463]:
holidays = [datetime.date(2014, 1, 1), datetime.date(2014, 5, 26), datetime.date(2014, 7, 4), datetime.date(2014, 9, 1),
           datetime.date(2014, 11, 27), datetime.date(2014, 12, 25)]
weekday = bainbridge_departures_2013_with_index["2013-01-02"].index + pd.offsets.DateOffset(years=1)
saturday = bainbridge_departures_2013_with_index["2013-01-05"].index + pd.offsets.DateOffset(years=1)
sunday = bainbridge_departures_2013_with_index["2013-01-06"].index + pd.offsets.DateOffset(years=1)

startingDate = datetime.date(2014, 1, 1)

In [464]:
timestamps_2014 = generate_more_timestamps(startingDate, holidays, weekday, saturday, sunday)

In [465]:
len(timestamps_2014)

8234

In [466]:
# Dropping 2013
bainbridge_departures_n13 = bainbridge_departures[8234:]

In [467]:
bainbridge_departures_n13 = bainbridge_departures_n13.reset_index(drop=True)

In [468]:
bainbridge_departures_n13.head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
0,Bainbridge,Colman,55:00.0,13:39.0,46:53.0,33.0
1,Bainbridge,Colman,20:00.0,20:00.0,55:34.0,35.0
2,Bainbridge,Colman,05:00.0,04:34.0,35:51.0,31.0
3,Bainbridge,Colman,55:00.0,55:00.0,26:14.0,31.0
4,Bainbridge,Colman,45:00.0,45:00.0,14:54.0,29.0


In [469]:
bainbridge_departures_2014_with_index = bainbridge_departures_n13[:8235]
bainbridge_departures_2014_with_index.tail()
#bainbridge_departures_2013_with_index.index = pd.to_datetime(timestamps_2014)

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
8230,Bainbridge,Colman,10:00.0,11:05.0,41:10.0,30.0
8231,Bainbridge,Colman,55:00.0,55:10.0,26:03.0,31.0
8232,Bainbridge,Colman,45:00.0,47:13.0,19:07.0,32.0
8233,Bainbridge,Colman,35:00.0,39:34.0,11:57.0,32.0
8234,Bainbridge,Colman,55:00.0,05:15.0,36:38.0,31.0


In [470]:
bainbridge_departures_2014_with_index = bainbridge_departures_2014_with_index.drop(2763)
bainbridge_departures_2014_with_index = bainbridge_departures_2014_with_index.reset_index(drop=True)

In [471]:
def verify_timestamps(dataset, indicies):
    for index, row in dataset.iterrows():
        scheduled_minute = indicies[index].minute
        dataset_minute = extractMinutesPastHour(row.iloc[2])
        
        if(not scheduled_minute == dataset_minute):    
            print("""Was able to match departure times up until this row at index %d:\n%s\n
                    Was trying to insert this timestamp: %s""" % 
                                              (index, row, indicies[index]))
            break

In [472]:
verify_timestamps(bainbridge_departures_2014_with_index, timestamps_2014)

In [473]:
len(timestamps_2014)

8234

In [474]:
bainbridge_departures_2014_with_index.shape

(8234, 6)

In [475]:
bainbridge_departures_2014_with_index.index = pd.to_datetime(timestamps_2014)
bainbridge_departures_2014_with_index = bainbridge_departures_2014_with_index["2014"]

In [476]:
bainbridge_departures_2014_with_index.head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
2014-01-01 00:55:00,Bainbridge,Colman,55:00.0,13:39.0,46:53.0,33.0
2014-01-01 05:20:00,Bainbridge,Colman,20:00.0,20:00.0,55:34.0,35.0
2014-01-01 07:05:00,Bainbridge,Colman,05:00.0,04:34.0,35:51.0,31.0
2014-01-01 07:55:00,Bainbridge,Colman,55:00.0,55:00.0,26:14.0,31.0
2014-01-01 08:45:00,Bainbridge,Colman,45:00.0,45:00.0,14:54.0,29.0


In [477]:
bainbridge_departures_2014_with_index.tail()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
2014-12-31 19:10:00,Bainbridge,Colman,10:00.0,16:19.0,48:52.0,32.0
2014-12-31 20:10:00,Bainbridge,Colman,10:00.0,11:05.0,41:10.0,30.0
2014-12-31 20:55:00,Bainbridge,Colman,55:00.0,55:10.0,26:03.0,31.0
2014-12-31 21:45:00,Bainbridge,Colman,45:00.0,47:13.0,19:07.0,32.0
2014-12-31 22:35:00,Bainbridge,Colman,35:00.0,39:34.0,11:57.0,32.0


In [480]:
bainbridge_departures_n13.iloc[8234]

departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       55:00.0
actual_departure          05:15.0
Actual_Arrival            36:38.0
TransitTime                    31
Name: 8234, dtype: object

In [483]:
bainbridge_departures_n13_n14 = bainbridge_departures_n13.iloc[8234:].reset_index(drop=True)

Here is the start

In [484]:
bainbridge_departures_n13_n14.head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
0,Bainbridge,Colman,55:00.0,05:15.0,36:38.0,31.0
1,Bainbridge,Colman,20:00.0,21:48.0,51:23.0,30.0
2,Bainbridge,Colman,05:00.0,06:20.0,36:29.0,30.0
3,Bainbridge,Colman,55:00.0,55:00.0,27:16.0,32.0
4,Bainbridge,Colman,45:00.0,45:55.0,15:02.0,30.0
